In [7]:
from el_toolkit.document import Document
from el_toolkit.data_processors import segment_document
from el_toolkit.data_displayer import Displayer
from transformers import BertTokenizer
from IPython.display import display, HTML

biomed_tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path='monologg/biobert_v1.1_pubmed',
                                            do_lower_case=False, cache_dir=None)

test_doc = Document.read_json("TREND_repos/BioMedical-EL/tests/test_data/test_docs/without_overlaps/17_mentions.json")

display(HTML(Displayer.display(test_doc)))
for doc in segment_document(test_doc,biomed_tokenizer,8):
  display(HTML(Displayer.display(doc)))
  print("-----------------------------------")


Mentions Omitted: 0


NameError: name 'displayHTML' is not defined